In [1]:
import numpy as np
from skimage import io
from skimage.transform import rescale, resize

import torch
from transformers import Dinov2Backbone

from torchvision.models import resnet18

/home/iternal/miniconda3/envs/rerun/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/iternal/miniconda3/envs/rerun/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# compute difference between crops



def main():
    # model_ = torch.load("weights/dinov2_s_cow_front_face_fine_tuned_model.pth")
    model = Dinov2Backbone.from_pretrained("facebook/dinov2-base")

    # pixel_values = torch.randn(1, 3, 224, 224)
    #
    # outputs = model(pixel_values)
    #
    # for feature_map in outputs.feature_maps:
    #     print(feature_map.shape)

    imginfo = lambda img: print(type(img), img.dtype, img.shape, img.min(), img.max())

    first_image = io.imread("first_image.jpg")
    max_image = io.imread("max_image.jpg")

    diff = first_image - max_image
    diff = np.sum(diff ** 2, axis=2) ** 0.5 * 10
    imginfo(diff)

    io.imsave("result.jpg", diff.astype(np.uint8))

    def to_batch(image):
        downscaled = resize(image, (224, 224))
        batch = torch.from_numpy(downscaled)[None, :, :, :]  # [h, w, c] -> [1, h, w, c]
        batch = batch.permute(0, 3, 1, 2)  # [1, h, w, c] -> [1, c, h, w]
        return batch

    def crop(image):
        """
        :param image: [h, w, c]
        """
        x, y = 900, 240
        cropw = 200
        return image[y-cropw:y+cropw, x-cropw:x+cropw]

    io.imsave("result_crop_a.jpg", crop(first_image).astype(np.uint8))
    io.imsave("result_crop_b.jpg", crop(max_image).astype(np.uint8))

    batch = to_batch(crop(first_image))
    imginfo(batch)
    dino1 = model(batch).feature_maps[0].detach()

    dino2 = model(to_batch(crop(max_image))).feature_maps[0].detach()


    imginfo(dino1)
    imginfo(dino2)

    diff = dino1 - dino2
    diff = (diff[0, :, :, :] ** 2).sum(axis=0)
    diff = diff.numpy()

    def scale(diff_image: np.array):
        low, high = np.quantile(diff, 0.1), np.quantile(diff, 0.9)
        return np.clip(diff_image / high * 255, 0, 255)

    diff = scale(diff)
    io.imsave("result.jpg", resize(diff, (400, 400)).astype(np.uint8))

main()

/home/iternal/miniconda3/envs/rerun/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


<class 'numpy.ndarray'> float64 (720, 1280) 0.0 273.31300737432895
<class 'torch.Tensor'> torch.float64 torch.Size([1, 3, 224, 224]) tensor(0.0003, dtype=torch.float64) tensor(0.9991, dtype=torch.float64)


2024-07-09 17:13:34.338365: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-09 17:13:34.339596: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-09 17:13:34.343903: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-09 17:13:34.351092: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-09 17:13:34.365113: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registe

<class 'torch.Tensor'> torch.float32 torch.Size([1, 768, 16, 16]) tensor(-15.1961) tensor(25.4411)
<class 'torch.Tensor'> torch.float32 torch.Size([1, 768, 16, 16]) tensor(-15.6322) tensor(22.3742)


In [2]:
backbone = resnet18(pretrained=True)

/home/iternal/miniconda3/envs/rerun/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/iternal/miniconda3/envs/rerun/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [13]:
imginfo = lambda img: print(type(img), img.dtype, img.shape, img.min(), img.max())

first_image = io.imread("first_image.jpg")
max_image = io.imread("max_image.jpg")

def to_batch(image):
    downscaled = resize(image, (224, 224))
    batch = torch.from_numpy(downscaled)[None, :, :, :].to(torch.float32) / 255  # [h, w, c] -> [1, h, w, c]
    batch = batch.permute(0, 3, 1, 2)  # [1, h, w, c] -> [1, c, h, w]
    return batch

def crop(image):
    """
    :param image: [h, w, c]
    """
    x, y = 830, 335
    cropw = 200
    return image[y-cropw:y+cropw, x-cropw:x+cropw]

io.imsave("result_crop_a.jpg", crop(first_image).astype(np.uint8))
io.imsave("result_crop_b.jpg", crop(max_image).astype(np.uint8))

In [14]:
batch = to_batch(crop(first_image))
imginfo(batch)

<class 'torch.Tensor'> torch.float32 torch.Size([1, 3, 224, 224]) tensor(0.) tensor(0.0039)


In [15]:
children = list(backbone.children())
newmodel = torch.nn.Sequential(*children[0:6])
newmodel

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [16]:
res = newmodel(batch).detach()

In [17]:
imginfo(res)

<class 'torch.Tensor'> torch.float32 torch.Size([1, 128, 28, 28]) tensor(0.) tensor(4.8947)


In [18]:
batch = to_batch(crop(first_image))
imginfo(batch)
dino1 = newmodel(batch).detach()
dino2 = newmodel(to_batch(crop(max_image))).detach()


imginfo(dino1)
imginfo(dino2)

diff = dino1 - dino2
diff = (diff[0, :, :, :] ** 2).sum(axis=0)
diff = diff.numpy()

def scale(diff_image: np.array):
    low, high = np.quantile(diff, 0.1), np.quantile(diff, 0.9)
    return np.clip(diff_image / high * 255, 0, 255)

diff = scale(diff)
io.imsave("result.jpg", resize(diff, (400, 400)).astype(np.uint8))

<class 'torch.Tensor'> torch.float32 torch.Size([1, 3, 224, 224]) tensor(0.) tensor(0.0039)
<class 'torch.Tensor'> torch.float32 torch.Size([1, 128, 28, 28]) tensor(0.) tensor(4.8947)
<class 'torch.Tensor'> torch.float32 torch.Size([1, 128, 28, 28]) tensor(0.) tensor(6.1370)
